## Iteration 4

In the fourth iteration, it was decided to allow the model to work with arbitrary input and to switch from Generator-only mode to a Solver as well. 

The idea was to update only the MathTutor class. It should decide right now whether it sees a question or a task request, and how to act. The resulting output is the same: either a new task or the user request reformulated as a task, so we do not need to update the downstream Judge. 

In [88]:
import json
import jsonlines
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [89]:
pd.set_option('display.max_colwidth', None)

In [90]:
load_dotenv()

True

In [91]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'type': 'task',
 'description': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': '1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \\times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.',
 'answer': '20'}

In [92]:
generator_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
judge_model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [93]:
class MathTutor:
    def __init__(self, model):
        self.model = model
        self.system_prompt = """Ти - вчитель математики. Проаналізуй запит користувача та надай відповідь.

Якщо користувач вказав тип контенту (Задача/Тест) - використай його. Якщо ні - обери відповідний тип самостійно.

Типи контенту:
1. Задача: 
   - description: умова задачі з одним питанням
   - solution: нумерований список кроків розв'язання
   - answer: число або вираз (без додатковий позначень)

2. Тест:
   - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)
   - solution: нумерований список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)

Відповідь надай у JSON форматі з чотирма полями: type (Задача або Тест), description, solution, answer.
Не обгортай JSON у додаткові спецсимволи."""
        self.generation_example = {
            'type': 'Задача',
            'description': 'Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\pi \\approx 3.14$.',
            'solution': '1. Згадайте формулу площі круга: $S = \\pi r^2$. 2. Підставте значення радіуса та числа $\\pi$: $S \\approx 3.14 \\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\cdot 16 = 50.24$.',
            'answer': '50.24'
        }
        self.solver_task_example = {
            'type': 'Задача',
            'description': 'Знайдіть корінь рівняння $2x - 5 = 7$.',
            'solution': '1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте відповідь: $x = 6$.',
            'answer': '6'
        }
        self.solver_quiz_example = {
            'type': 'Тест',
            'description': 'Чому дорівнює $\\sqrt{144}$? \\n A) 10 \\n B) 11 \\n C) 12 \\n D) 14',
            'solution': '1. Визначте, що потрібно знайти число, яке при множенні саме на себе дає 144. 2. Перевірте варіанти відповідей піднесенням до квадрату. 3. $10^2=100$, $11^2=121$, $12^2=144$. 4. Оскільки $12^2=144$, оберіть варіант C.',
            'answer': 'C'
        }

    def invoke(self, user_input):
        generation_json = json.dumps(self.generation_example, ensure_ascii=False)
        solver_task_json = json.dumps(self.solver_task_example, ensure_ascii=False)
        solver_quiz_json = json.dumps(self.solver_quiz_example, ensure_ascii=False)
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content="Тема: Площа круга, Тип: Задача"),
            AIMessage(content=generation_json),
            HumanMessage(content="Знайдіть корінь рівняння $2x - 5 = 7$."),
            AIMessage(content=solver_task_json),
            HumanMessage(content="Чому дорівнює $\\sqrt{144}$? \\n A) 10 \\n B) 11 \\n C) 12 \\n D) 14"),
            AIMessage(content=solver_quiz_json),
            HumanMessage(content=user_input)
        ]
        response = self.model.invoke(messages)
        return response.content

In [94]:
class AcceptanceJudge:
    def __init__(self, model):
        self.model = model
        self.system_prompt = """Ти - експерт з оцінювання математичного контенту. Оціни наданий контент за критеріями.

Критерії:
1. has_numbered_solution: чи містить поле solution нумерований список кроків (1. ... 2. ... тощо)
2. has_valid_answer_format: 
   - для типу "Задача": відповідь має бути числом або виразом, або їх комбінацією
   - для типу "Тест": відповідь має бути однією латинською літерою (A, B, C або D)
3. has_valid_description_format:
   - для типу "Задача": опис має містити умову задачі
   - для типу "Тест": опис має містити чотири варіанти відповіді (A, B, C, D)

Відповідь надай у JSON форматі з трьома булевими полями: has_numbered_solution, has_valid_answer_format, has_valid_description_format. 
У разі якщо хоча б одне поле не є true, додай четверте поле reason з описом чому.
Не обгортай JSON у додаткові спецсимволи."""
        self.task_example = {
            'description': 'Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\pi \\approx 3.14$.',
            'solution': '1. Згадайте формулу площі круга: $S = \\pi r^2$. 2. Підставте значення радіуса та числа $\\pi$: $S \\approx 3.14 \\cdot 4^2$. 3. Піднесіть радіус до квадрата: $4^2 = 16$. 4. Виконайте множення: $3.14 \\cdot 16 = 50.24$.',
            'answer': '50.24'
        }
        self.quiz_example = {
            'description': 'Знайдіть корінь рівняння $2x - 5 = 7$. \n A) 1 \n B) 6 \n C) 12 \n D) 2',
            'solution': '1. Перенесіть вільний член (-5) у праву частину рівняння, змінивши знак на протилежний: $2x = 7 + 5$. 2. Обчисліть суму в правій частині: $2x = 12$. 3. Поділіть обидві частини рівняння на коефіцієнт при змінній (на 2): $x = 12 / 2$. 4. Отримайте $x = 6$ і оберіть відповідний варіант B.',
            'answer': 'B'
        }

    def evaluate(self, content, content_type):
        content_json = json.dumps(content, ensure_ascii=False)
        task_example_json = json.dumps(self.task_example, ensure_ascii=False)
        quiz_example_json = json.dumps(self.quiz_example, ensure_ascii=False)
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=f"Тип контенту: Задача\n\nКонтент:\n{task_example_json}"),
            AIMessage(content='{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'),
            HumanMessage(content=f"Тип контенту: Тест\n\nКонтент:\n{quiz_example_json}"),
            AIMessage(content='{"has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'),
            HumanMessage(content=f"Тип контенту: {content_type}\n\nКонтент:\n{content_json}")
        ]
        response = self.model.invoke(messages)
        return response.content

In [95]:
tutor = MathTutor(generator_model)
judge = AcceptanceJudge(judge_model)

results = []

for item in dataset[:5]:

    user_input = f"Тема: {item['topic']}"
    generated_content = tutor.invoke(user_input)
    
    try:
        parsed = json.loads(generated_content)
        is_valid_json = True
        generated_type = parsed.get('type', 'Задача')
        generated_description = parsed.get('description', '')
        generated_solution = parsed.get('solution', '')
        generated_answer = parsed.get('answer', '')
    except json.JSONDecodeError:
        is_valid_json = False
        parsed = {}
        generated_type = 'Задача'
        generated_description = ''
        generated_solution = ''
        generated_answer = ''

    if is_valid_json:
        judge_response = judge.evaluate(parsed, generated_type)
        try:
            judge_parsed = json.loads(judge_response)
            judge_valid_json = True
            has_numbered_solution = judge_parsed.get('has_numbered_solution', False)
            has_valid_answer_format = judge_parsed.get('has_valid_answer_format', False)
            has_valid_description_format = judge_parsed.get('has_valid_description_format', False)
        except json.JSONDecodeError:
            judge_valid_json = False
            has_numbered_solution = False
            has_valid_answer_format = False
            has_valid_description_format = False
    else:
        judge_response = ''
        judge_valid_json = False
        has_numbered_solution = False
        has_valid_answer_format = False
        has_valid_description_format = False
    
    display(pd.DataFrame([
        {'field': 'mode', 'value': 'generation'},
        {'field': 'user_input', 'value': user_input},
        {'field': 'generated_content', 'value': generated_content},
        {'field': 'is_valid_json', 'value': is_valid_json},
        {'field': 'generated_type', 'value': generated_type},
        {'field': 'generated_description', 'value': generated_description},
        {'field': 'generated_solution', 'value': generated_solution},
        {'field': 'generated_answer', 'value': generated_answer},
        {'field': 'judge_response', 'value': judge_response},
        {'field': 'judge_valid_json', 'value': judge_valid_json},
        {'field': 'has_numbered_solution', 'value': has_numbered_solution},
        {'field': 'has_valid_answer_format', 'value': has_valid_answer_format},
        {'field': 'has_valid_description_format', 'value': has_valid_description_format},
    ]))

    results.append({
        'mode': 'generation',
        'user_input': user_input,
        'generated_type': generated_type,
        'is_valid_json': is_valid_json,
        'judge_valid_json': judge_valid_json,
        'has_numbered_solution': has_numbered_solution,
        'has_valid_answer_format': has_valid_answer_format,
        'has_valid_description_format': has_valid_description_format,
    })

for item in dataset[:5]:

    user_input = item['description']
    generated_content = tutor.invoke(user_input)
    
    try:
        parsed = json.loads(generated_content)
        is_valid_json = True
        generated_type = parsed.get('type', 'Задача')
        generated_description = parsed.get('description', '')
        generated_solution = parsed.get('solution', '')
        generated_answer = parsed.get('answer', '')
    except json.JSONDecodeError:
        is_valid_json = False
        parsed = {}
        generated_type = 'Задача'
        generated_description = ''
        generated_solution = ''
        generated_answer = ''

    if is_valid_json:
        judge_response = judge.evaluate(parsed, generated_type)
        try:
            judge_parsed = json.loads(judge_response)
            judge_valid_json = True
            has_numbered_solution = judge_parsed.get('has_numbered_solution', False)
            has_valid_answer_format = judge_parsed.get('has_valid_answer_format', False)
            has_valid_description_format = judge_parsed.get('has_valid_description_format', False)
        except json.JSONDecodeError:
            judge_valid_json = False
            has_numbered_solution = False
            has_valid_answer_format = False
            has_valid_description_format = False
    else:
        judge_response = ''
        judge_valid_json = False
        has_numbered_solution = False
        has_valid_answer_format = False
        has_valid_description_format = False
    
    display(pd.DataFrame([
        {'field': 'mode', 'value': 'solver'},
        {'field': 'user_input', 'value': user_input},
        {'field': 'expected_answer', 'value': item['answer']},
        {'field': 'generated_content', 'value': generated_content},
        {'field': 'is_valid_json', 'value': is_valid_json},
        {'field': 'generated_type', 'value': generated_type},
        {'field': 'generated_description', 'value': generated_description},
        {'field': 'generated_solution', 'value': generated_solution},
        {'field': 'generated_answer', 'value': generated_answer},
        {'field': 'judge_response', 'value': judge_response},
        {'field': 'judge_valid_json', 'value': judge_valid_json},
        {'field': 'has_numbered_solution', 'value': has_numbered_solution},
        {'field': 'has_valid_answer_format', 'value': has_valid_answer_format},
        {'field': 'has_valid_description_format', 'value': has_valid_description_format},
    ]))

    results.append({
        'mode': 'solver',
        'user_input': user_input[:50] + '...',
        'generated_type': generated_type,
        'is_valid_json': is_valid_json,
        'judge_valid_json': judge_valid_json,
        'has_numbered_solution': has_numbered_solution,
        'has_valid_answer_format': has_valid_answer_format,
        'has_valid_description_format': has_valid_description_format,
    })

results = pd.DataFrame(results)

,field,value
0,mode,generation
1,user_input,Тема: Арифметичні дії
2,generated_content,"{""type"": ""Задача"", ""description"": ""Обчисліть результат виразу $8 + 2 \times 5 - 3^2$."", ""solution"": ""1. Виконайте дії у дужках (якщо є) - у цьому виразі дужок немає. 2. Виконайте піднесення до степеня: $3^2 = 9$. 3. Виконайте множення: $2 \\times 5 = 10$. 4. Розв'яжіть вираз, виконуючи дії зліва направо: $8 + 10 - 9$. 5. Додайте: $8 + 10 = 18$. 6. Відніміть: $18 - 9 = 9$."", ""answer"": ""9""}"
3,is_valid_json,True
4,generated_type,Задача
5,generated_description,Обчисліть результат виразу $8 + 2 \times 5 - 3^2$.
6,generated_solution,"1. Виконайте дії у дужках (якщо є) - у цьому виразі дужок немає. 2. Виконайте піднесення до степеня: $3^2 = 9$. 3. Виконайте множення: $2 \times 5 = 10$. 4. Розв'яжіть вираз, виконуючи дії зліва направо: $8 + 10 - 9$. 5. Додайте: $8 + 10 = 18$. 6. Відніміть: $18 - 9 = 9$."
7,generated_answer,9
8,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"
9,judge_valid_json,True


,field,value
0,mode,generation
1,user_input,Тема: Геометрія
2,generated_content,"{""type"": ""Задача"", ""description"": ""Знайдіть довжину діагоналі прямокутника, якщо його сторони дорівнюють 6 см і 8 см."", ""solution"": ""1. Згадайте теорему Піфагора для прямокутника, де діагональ є гіпотенузою: $d^2 = a^2 + b^2$, де $d$ - діагональ, $a$ і $b$ - сторони прямокутника. 2. Підставте значення сторін: $d^2 = 6^2 + 8^2$. 3. Обчисліть квадрати сторін: $6^2 = 36$ і $8^2 = 64$. 4. Складіть результати: $d^2 = 36 + 64 = 100$. 5. Знайдіть діагональ, взявши квадратний корінь: $d = \\sqrt{100} = 10$ см."", ""answer"": ""10""}"
3,is_valid_json,True
4,generated_type,Задача
5,generated_description,"Знайдіть довжину діагоналі прямокутника, якщо його сторони дорівнюють 6 см і 8 см."
6,generated_solution,"1. Згадайте теорему Піфагора для прямокутника, де діагональ є гіпотенузою: $d^2 = a^2 + b^2$, де $d$ - діагональ, $a$ і $b$ - сторони прямокутника. 2. Підставте значення сторін: $d^2 = 6^2 + 8^2$. 3. Обчисліть квадрати сторін: $6^2 = 36$ і $8^2 = 64$. 4. Складіть результати: $d^2 = 36 + 64 = 100$. 5. Знайдіть діагональ, взявши квадратний корінь: $d = \sqrt{100} = 10$ см."
7,generated_answer,10
8,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"
9,judge_valid_json,True


,field,value
0,mode,generation
1,user_input,Тема: Спрощення виразів
2,generated_content,"{""type"": ""Задача"", ""description"": ""Спростіть вираз $3(x + 4) - 2(x - 5)$."", ""solution"": ""1. Розкрийте дужки, застосовуючи розподільний закон: $3(x + 4) = 3x + 12$ і $-2(x - 5) = -2x + 10$. 2. Замініть вихідний вираз: $3x + 12 - 2x + 10$. 3. Зведіть подібні доданки: $3x - 2x = x$ і $12 + 10 = 22$. 4. Отримайте спрощений вираз: $x + 22$."", ""answer"": ""x + 22""}"
3,is_valid_json,True
4,generated_type,Задача
5,generated_description,Спростіть вираз $3(x + 4) - 2(x - 5)$.
6,generated_solution,"1. Розкрийте дужки, застосовуючи розподільний закон: $3(x + 4) = 3x + 12$ і $-2(x - 5) = -2x + 10$. 2. Замініть вихідний вираз: $3x + 12 - 2x + 10$. 3. Зведіть подібні доданки: $3x - 2x = x$ і $12 + 10 = 22$. 4. Отримайте спрощений вираз: $x + 22$."
7,generated_answer,x + 22
8,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"
9,judge_valid_json,True


,field,value
0,mode,generation
1,user_input,Тема: Нерівності
2,generated_content,"{""type"": ""Задача"", ""description"": ""Розв’яжіть нерівність $3x - 4 < 5$"", ""solution"": ""1. Додайте 4 до обох частин нерівності, щоб позбутися від -4: $3x - 4 + 4 < 5 + 4$. 2. Отримайте: $3x < 9$. 3. Поділіть обидві частини нерівності на 3, щоб знайти $x$: $x < 9 / 3$. 4. Обчисліть: $x < 3$."", ""answer"": ""x < 3""}"
3,is_valid_json,True
4,generated_type,Задача
5,generated_description,Розв’яжіть нерівність $3x - 4 < 5$
6,generated_solution,"1. Додайте 4 до обох частин нерівності, щоб позбутися від -4: $3x - 4 + 4 < 5 + 4$. 2. Отримайте: $3x < 9$. 3. Поділіть обидві частини нерівності на 3, щоб знайти $x$: $x < 9 / 3$. 4. Обчисліть: $x < 3$."
7,generated_answer,x < 3
8,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"
9,judge_valid_json,True


,field,value
0,mode,generation
1,user_input,Тема: Системи рівнянь
2,generated_content,"{""type"": ""Задача"", ""description"": ""Розв'яжіть систему рівнянь: \\( \\begin{cases} 2x + y = 10 \\\\ 3x - 2y = 1 \\end{cases} \\)."", ""solution"": ""1. Виберіть метод для розв'язання системи, наприклад, метод підстановки або метод додавання. Використаємо метод додавання. 2. Помножте перше рівняння на 2, щоб отримати однакові коефіцієнти перед \\(y\\): \\(4x + 2y = 20\\). 3. Складіть рівняння з другим: \\((4x + 2y) + (3x - 2y) = 20 + 1\\). 4. Спрощення дає: \\(7x = 21\\). 5. Поділіть обидві частини на 7: \\(x = 3\\). 6. Підставте знайдене \\(x\\) у перше рівняння: \\(2(3) + y = 10\\). 7. Обчисліть \\(y\\): \\(6 + y = 10\\) дає \\(y = 4\\)."", ""answer"": ""x = 3, y = 4""}"
3,is_valid_json,True
4,generated_type,Задача
5,generated_description,Розв'яжіть систему рівнянь: \( \begin{cases} 2x + y = 10 \\ 3x - 2y = 1 \end{cases} \).
6,generated_solution,"1. Виберіть метод для розв'язання системи, наприклад, метод підстановки або метод додавання. Використаємо метод додавання. 2. Помножте перше рівняння на 2, щоб отримати однакові коефіцієнти перед \(y\): \(4x + 2y = 20\). 3. Складіть рівняння з другим: \((4x + 2y) + (3x - 2y) = 20 + 1\). 4. Спрощення дає: \(7x = 21\). 5. Поділіть обидві частини на 7: \(x = 3\). 6. Підставте знайдене \(x\) у перше рівняння: \(2(3) + y = 10\). 7. Обчисліть \(y\): \(6 + y = 10\) дає \(y = 4\)."
7,generated_answer,"x = 3, y = 4"
8,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"
9,judge_valid_json,True


,field,value
0,mode,solver
1,user_input,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
2,expected_answer,20
3,generated_content,"{""type"": ""Задача"", ""description"": ""Обчисліть значення виразу: $100 - (15 + 5) \\times 4$."", ""solution"": ""1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Помножте результат на 4: $20 \\times 4 = 80$. 3. Відніміть отримане значення від 100: $100 - 80 = 20$."", ""answer"": ""20""}"
4,is_valid_json,True
5,generated_type,Задача
6,generated_description,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
7,generated_solution,1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Помножте результат на 4: $20 \times 4 = 80$. 3. Відніміть отримане значення від 100: $100 - 80 = 20$.
8,generated_answer,20
9,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"


,field,value
0,mode,solver
1,user_input,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
2,expected_answer,B
3,generated_content,"{""type"": ""Тест"", ""description"": ""Чому дорівнює сума внутрішніх кутів довільного трикутника? \\n A) $90^\\circ$ \\n B) $180^\\circ$ \\n C) $270^\\circ$ \\n D) $360^\\circ$"", ""solution"": ""1. Згадайте теорему про суму кутів трикутника, яка стверджує, що сума внутрішніх кутів будь-якого трикутника дорівнює $180^\\circ$. 2. Перевірте варіанти відповідей. 3. Виберіть правильний варіант, який відповідає теоремі."", ""answer"": ""B""}"
4,is_valid_json,True
5,generated_type,Тест
6,generated_description,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
7,generated_solution,"1. Згадайте теорему про суму кутів трикутника, яка стверджує, що сума внутрішніх кутів будь-якого трикутника дорівнює $180^\circ$. 2. Перевірте варіанти відповідей. 3. Виберіть правильний варіант, який відповідає теоремі."
8,generated_answer,B
9,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"


,field,value
0,mode,solver
1,user_input,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
2,expected_answer,$x^2 - 9$
3,generated_content,"{""type"": ""Задача"", ""description"": ""Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."", ""solution"": ""1. Згадайте формулу різниці квадратів: $(a - b)(a + b) = a^2 - b^2$. 2. В даному випадку $a = x$ і $b = 3$. 3. Застосуйте формулу: $(x - 3)(x + 3) = x^2 - 3^2$. 4. Обчисліть $3^2 = 9$. 5. Отримайте спрощений вираз: $x^2 - 9$."", ""answer"": ""x^2 - 9""}"
4,is_valid_json,True
5,generated_type,Задача
6,generated_description,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
7,generated_solution,1. Згадайте формулу різниці квадратів: $(a - b)(a + b) = a^2 - b^2$. 2. В даному випадку $a = x$ і $b = 3$. 3. Застосуйте формулу: $(x - 3)(x + 3) = x^2 - 3^2$. 4. Обчисліть $3^2 = 9$. 5. Отримайте спрощений вираз: $x^2 - 9$.
8,generated_answer,x^2 - 9
9,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"


,field,value
0,mode,solver
1,user_input,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
2,expected_answer,C
3,generated_content,"{""type"": ""Тест"", ""description"": ""Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\\infty; 5)$ \n B) $[5; +\\infty)$ \n C) $(5; +\\infty)$ \n D) $(-\\infty; 5]$"", ""solution"": ""1. Розгляньте нерівність $x > 5$, яка означає, що $x$ більше ніж 5. 2. Проміжок $(-\\infty; 5)$ включає числа менші за 5, що не є розв'язком. 3. Проміжок $[5; +\\infty)$ включає число 5, але $x$ не може дорівнювати 5. 4. Проміжок $(5; +\\infty)$ включає числа, що більше 5, але не включає 5, що є правильним розв'язком. 5. Проміжок $(-\\infty; 5]$ включає числа менші або рівні 5, що не є розв'язком."", ""answer"": ""C""}"
4,is_valid_json,True
5,generated_type,Тест
6,generated_description,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
7,generated_solution,"1. Розгляньте нерівність $x > 5$, яка означає, що $x$ більше ніж 5. 2. Проміжок $(-\infty; 5)$ включає числа менші за 5, що не є розв'язком. 3. Проміжок $[5; +\infty)$ включає число 5, але $x$ не може дорівнювати 5. 4. Проміжок $(5; +\infty)$ включає числа, що більше 5, але не включає 5, що є правильним розв'язком. 5. Проміжок $(-\infty; 5]$ включає числа менші або рівні 5, що не є розв'язком."
8,generated_answer,C
9,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"


,field,value
0,mode,solver
1,user_input,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
2,expected_answer,"$x = 7, y = 3$"
3,generated_content,"{""type"": ""Задача"", ""description"": ""Розв'яжіть систему рівнянь: $\\begin{cases} x + y = 10 \\\\ x - y = 4 \\end{cases}$."", ""solution"": ""1. Додайте обидва рівняння, щоб позбутися змінної $y$: $(x + y) + (x - y) = 10 + 4$. 2. Отримайте $2x = 14$. 3. Поділіть обидві частини на 2, щоб знайти $x$: $x = 7$. 4. Підставте $x = 7$ у перше рівняння: $7 + y = 10$. 5. Відніміть 7 від обох частин, щоб знайти $y$: $y = 3$."", ""answer"": ""(7, 3)""}"
4,is_valid_json,True
5,generated_type,Задача
6,generated_description,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$.
7,generated_solution,"1. Додайте обидва рівняння, щоб позбутися змінної $y$: $(x + y) + (x - y) = 10 + 4$. 2. Отримайте $2x = 14$. 3. Поділіть обидві частини на 2, щоб знайти $x$: $x = 7$. 4. Підставте $x = 7$ у перше рівняння: $7 + y = 10$. 5. Відніміть 7 від обох частин, щоб знайти $y$: $y = 3$."
8,generated_answer,"(7, 3)"
9,judge_response,"{""has_numbered_solution"": true, ""has_valid_answer_format"": true, ""has_valid_description_format"": true}"


In [96]:
display(results)

bool_columns = ['is_valid_json', 'judge_valid_json', 'has_numbered_solution', 'has_valid_answer_format', 'has_valid_description_format']
failures = results[~results[bool_columns].all(axis=1)]

print(f"Failures: {len(failures)} / {len(results)}")
if not failures.empty:
    display(failures)

,mode,user_input,generated_type,is_valid_json,judge_valid_json,has_numbered_solution,has_valid_answer_format,has_valid_description_format
0,generation,Тема: Арифметичні дії,Задача,True,True,True,True,True
1,generation,Тема: Геометрія,Задача,True,True,True,True,True
2,generation,Тема: Спрощення виразів,Задача,True,True,True,True,True
3,generation,Тема: Нерівності,Задача,True,True,True,True,True
4,generation,Тема: Системи рівнянь,Задача,True,True,True,True,True
5,solver,Обчисліть значення виразу: $100 - (15 + 5) \times ...,Задача,True,True,True,True,True
6,solver,Чому дорівнює сума внутрішніх кутів довільного три...,Тест,True,True,True,True,True
7,solver,"Спростіть вираз, використовуючи формулу різниці кв...",Задача,True,True,True,True,True
8,solver,Який проміжок є розв'язком нерівності $x > 5$? \n A...,Тест,True,True,True,True,True
9,solver,Розв'яжіть систему рівнянь: $\begin{cases} x + y =...,Задача,True,True,True,True,True


Failures: 0 / 10
